In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob 
import os
import warnings
import networkx as nx
import matplotlib.colors as mcolors
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy.stats import pearsonr
from sklearn.covariance import GraphicalLassoCV
from networkx.generators.community import gaussian_random_partition_graph
import scipy.stats as stats
from scipy.stats import ttest_ind
import nibabel as nib 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import tqdm

In [92]:
comb_gmv = pd.read_csv("/data2/MRI_PET_DATA/graph/csvs/graph_gmv_volumes/parcellation_volumes_raw.csv")

In [93]:
print(comb_gmv['Dataset'])

0       ADNI
1       ADNI
2       ADNI
3       ADNI
4       ADNI
5       ADNI
6       ADNI
7       ADNI
8       ADNI
9       ADNI
10      ADNI
11      ADNI
12      ADNI
13      ADNI
14      ADNI
15      ADNI
16      ADNI
17      ADNI
18      ADNI
19      ADNI
20      ADNI
21      ADNI
22      ADNI
23      ADNI
24      ADNI
25      ADNI
26      ADNI
27      ADNI
28      ADNI
29      ADNI
30      ADNI
31      ADNI
32      ADNI
33      ADNI
34      ADNI
35      ADNI
36      ADNI
37      ADNI
38      ADNI
39      ADNI
40      ADNI
41      ADNI
42      ADNI
43      ADNI
44      ADNI
45      ADNI
46      ADNI
47      ADNI
48      ADNI
49      ADNI
50      ADNI
51      ADNI
52      ADNI
53      ADNI
54      ADNI
55      ADNI
56      ADNI
57      ADNI
58      ADNI
59      ADNI
60      ADNI
61      ADNI
62      ADNI
63      ADNI
64      ADNI
65      ADNI
66      ADNI
67      ADNI
68      ADNI
69      ADNI
70      ADNI
71      ADNI
72      ADNI
73      ADNI
74      ADNI
75      ADNI
76      ADNI

In [94]:
#need to separate these into adni and nacc (more effcient to do a boolean mask but this is good bc I wrote it)
comb_gmv_adni = pd.DataFrame()

comb_gmv_nacc = pd.DataFrame()

for index,row in comb_gmv.iterrows():
    dataset = row['Dataset']
    if dataset == 'ADNI':
        comb_gmv_adni = comb_gmv_adni.append(row,ignore_index=True)
    if dataset == 'NACC':
        comb_gmv_nacc = comb_gmv_nacc.append(row,ignore_index=True)

print(comb_gmv_adni)

      RID          TIV  vol_l3thVen  vol_r3thVen  vol_l4thVen  vol_r4thVen  \
0    0173  1575.972878     0.022812     0.009325     0.056612     0.087529   
1    0307  1465.938133     0.029220     0.023451     0.045906     0.055573   
2    0331  1465.246157     0.039467     0.042651     0.037224     0.039259   
3    0376  1586.856867     0.019365     0.012784     0.047067     0.061749   
4    0388  1721.549255     0.027549     0.016580     0.080494     0.065424   
5    0397  1574.501525     0.042252     0.039029     0.045227     0.042648   
6    0417  1908.089286     0.012404     0.008631     0.029659     0.051816   
7    0423  1445.325235     0.026459     0.031231     0.081839     0.107271   
8    0429  1634.913239     0.058192     0.036102     0.089212     0.105514   
9    0442  1492.483220     0.025769     0.012525     0.077486     0.071035   
10   0448  1599.567886     0.029050     0.030147     0.040796     0.051118   
11   0449  1269.082384     0.042776     0.023129     0.052796   

In [95]:
print(comb_gmv_nacc)

            RID          TIV  vol_l3thVen  vol_r3thVen  vol_l4thVen  \
0    NACC000511  1555.177037     0.020264     0.013338     0.053174   
1    NACC000531  1647.584787     0.067283     0.052440     0.099190   
2    NACC003196  1362.881871     0.022565     0.022337     0.051906   
3    NACC006009  1409.219319     0.051523     0.050538     0.053296   
4    NACC006420  1323.345298     0.062451     0.058506     0.084043   
5    NACC007377  1155.948705     0.042392     0.035341     0.033247   
6    NACC008987  1478.138655     0.021388     0.009847     0.054729   
7    NACC011143  1344.911732     0.019556     0.008075     0.036934   
8    NACC012666  1623.975651     0.040341     0.041047     0.105792   
9    NACC013585  1683.455180     0.022341     0.019690     0.081929   
10   NACC015412  1588.981310     0.038149     0.025824     0.093200   
11   NACC015899  1385.628810     0.011388     0.010145     0.103969   
12   NACC016502  1558.432298     0.040541     0.032090     0.095408   
13   N

In [96]:
#save csvs 
comb_gmv_adni.to_csv("/data2/MRI_PET_DATA/graph/csvs/graph_gmv_volumes/mike_gmv_adni_combo.csv")
comb_gmv_nacc.to_csv("/data2/MRI_PET_DATA/graph/csvs/graph_gmv_volumes/mike_gmv_nacc_combo.csv")

In [97]:
print(comb_gmv['RID'][1:10])

1    0307
2    0331
3    0376
4    0388
5    0397
6    0417
7    0423
8    0429
9    0442
Name: RID, dtype: object


In [98]:
#read other csvs 
adni_progs = pd.read_csv("/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/progressors_vol.csv")
adni_stable = pd.read_csv("/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/stable_vol.csv")

In [99]:
print(adni_stable['RID'][1:10])

1    307
2    397
3    417
4    449
5    478
6    505
7    579
8    607
9    613
Name: RID, dtype: int64


In [100]:
#stratify them into prog and stable for adni

#do this by cross referencing the stable dfs that I have before from other csv files 
adni_progs['RID'] = adni_progs["RID"].astype('int64')
adni_stable['RID']=adni_stable['RID'].astype('int64')
comb_gmv_adni['RID']=comb_gmv_adni['RID'].astype('int64')

rid_comb_gmv = set(comb_gmv_adni["RID"])
rid_adni_progs = set(adni_progs['RID'])
rid_adni_stab = set(adni_stable['RID'])


adni_gmv_prog = pd.DataFrame()
adni_gmv_stab = pd.DataFrame()


#itterows allows you to iterate through rows of a df (it returns an iterator which is a pairs of index and series (the data for that row) for each row)
for index, row in comb_gmv_adni.iterrows():
    rid = row["RID"]

    if rid in rid_adni_progs:
        #if that rid exists in adni_progs then append the row to adni_gmv
        adni_gmv_prog = adni_gmv_prog.append(row,ignore_index=True)
    else:  
        adni_gmv_stab = adni_gmv_stab.append(row,ignore_index=True) #if they arent prog they are stab 

print(adni_gmv_prog)

      RID          TIV  vol_l3thVen  vol_r3thVen  vol_l4thVen  vol_r4thVen  \
0     331  1465.246157     0.039467     0.042651     0.037224     0.039259   
1     376  1586.856867     0.019365     0.012784     0.047067     0.061749   
2     388  1721.549255     0.027549     0.016580     0.080494     0.065424   
3     423  1445.325235     0.026459     0.031231     0.081839     0.107271   
4     429  1634.913239     0.058192     0.036102     0.089212     0.105514   
5     442  1492.483220     0.025769     0.012525     0.077486     0.071035   
6     448  1599.567886     0.029050     0.030147     0.040796     0.051118   
7     476  1215.996580     0.031886     0.028839     0.047639     0.050165   
8     501  1415.356435     0.075804     0.058510     0.038333     0.053714   
9     507  1298.318686     0.032035     0.017035     0.077945     0.075063   
10    514  1812.790973     0.022592     0.005941     0.057278     0.054278   
11    604  1666.773184     0.073722     0.062710     0.055153   

In [101]:
print(adni_gmv_stab)

      RID          TIV  vol_l3thVen  vol_r3thVen  vol_l4thVen  vol_r4thVen  \
0     173  1575.972878     0.022812     0.009325     0.056612     0.087529   
1     307  1465.938133     0.029220     0.023451     0.045906     0.055573   
2     397  1574.501525     0.042252     0.039029     0.045227     0.042648   
3     417  1908.089286     0.012404     0.008631     0.029659     0.051816   
4     449  1269.082384     0.042776     0.023129     0.052796     0.052325   
5     478  1662.609925     0.068710     0.073631     0.067298     0.059263   
6     505  1359.525847     0.033780     0.027729     0.068024     0.069890   
7     579  1497.574016     0.096937     0.051208     0.070200     0.050706   
8     607  1281.711753     0.036863     0.016071     0.046239     0.040949   
9     613  1353.701290     0.064961     0.041678     0.052325     0.063537   
10    746  1363.107098     0.023631     0.014196     0.073600     0.083404   
11    792  1488.578922     0.031004     0.018686     0.065992   

In [102]:
#save csvs of mike gmv andi prog and adni stab
adni_gmv_prog.to_csv("/data2/MRI_PET_DATA/graph/csvs/graph_gmv_volumes/mike_gmv_adni_prog.csv", index=False)
adni_gmv_stab.to_csv('/data2/MRI_PET_DATA/graph/csvs/graph_gmv_volumes/mike_gmv_adni_stab.csv', index=False)
#add index = false when saving csvs 

In [103]:
# print(adni_gmv_stab) #looks good 
print(adni_gmv_prog.columns)

Index(['RID', 'TIV', 'vol_l3thVen', 'vol_r3thVen', 'vol_l4thVen',
       'vol_r4thVen', 'vol_lAcc', 'vol_rAcc', 'vol_lAmy', 'vol_rAmy',
       ...
       'vol_rSupParLo', 'vol_lSupTemGy', 'vol_rSupTemGy', 'vol_lTemPo',
       'vol_rTemPo', 'vol_lInfFroAngGy', 'vol_rInfFroAngGy', 'vol_lTemTraGy',
       'vol_rTemTraGy', 'Dataset'],
      dtype='object', length=145)


In [104]:
#now do the significance tests maybe
print(adni_gmv_prog)

      RID          TIV  vol_l3thVen  vol_r3thVen  vol_l4thVen  vol_r4thVen  \
0     331  1465.246157     0.039467     0.042651     0.037224     0.039259   
1     376  1586.856867     0.019365     0.012784     0.047067     0.061749   
2     388  1721.549255     0.027549     0.016580     0.080494     0.065424   
3     423  1445.325235     0.026459     0.031231     0.081839     0.107271   
4     429  1634.913239     0.058192     0.036102     0.089212     0.105514   
5     442  1492.483220     0.025769     0.012525     0.077486     0.071035   
6     448  1599.567886     0.029050     0.030147     0.040796     0.051118   
7     476  1215.996580     0.031886     0.028839     0.047639     0.050165   
8     501  1415.356435     0.075804     0.058510     0.038333     0.053714   
9     507  1298.318686     0.032035     0.017035     0.077945     0.075063   
10    514  1812.790973     0.022592     0.005941     0.057278     0.054278   
11    604  1666.773184     0.073722     0.062710     0.055153   

In [105]:
#normalize each by TIV
for column_name in adni_gmv_prog.columns:
    if column_name != 'TIV' and column_name !='RID' and column_name !='Dataset':
        adni_gmv_prog[column_name]= adni_gmv_prog[column_name]/adni_gmv_prog['TIV']

print(adni_gmv_prog)

      RID          TIV  vol_l3thVen  vol_r3thVen  vol_l4thVen  vol_r4thVen  \
0     331  1465.246157     0.000027     0.000029     0.000025     0.000027   
1     376  1586.856867     0.000012     0.000008     0.000030     0.000039   
2     388  1721.549255     0.000016     0.000010     0.000047     0.000038   
3     423  1445.325235     0.000018     0.000022     0.000057     0.000074   
4     429  1634.913239     0.000036     0.000022     0.000055     0.000065   
5     442  1492.483220     0.000017     0.000008     0.000052     0.000048   
6     448  1599.567886     0.000018     0.000019     0.000026     0.000032   
7     476  1215.996580     0.000026     0.000024     0.000039     0.000041   
8     501  1415.356435     0.000054     0.000041     0.000027     0.000038   
9     507  1298.318686     0.000025     0.000013     0.000060     0.000058   
10    514  1812.790973     0.000012     0.000003     0.000032     0.000030   
11    604  1666.773184     0.000044     0.000038     0.000033   

In [106]:
#normalize stable 
#normalize each by TIV
for column_name in adni_gmv_stab.columns:
    if column_name != 'TIV' and column_name !='RID' and column_name !='Dataset':
        adni_gmv_stab[column_name]= adni_gmv_stab[column_name]/adni_gmv_stab['TIV']

print(adni_gmv_stab)

      RID          TIV  vol_l3thVen  vol_r3thVen  vol_l4thVen  vol_r4thVen  \
0     173  1575.972878     0.000014     0.000006     0.000036     0.000056   
1     307  1465.938133     0.000020     0.000016     0.000031     0.000038   
2     397  1574.501525     0.000027     0.000025     0.000029     0.000027   
3     417  1908.089286     0.000007     0.000005     0.000016     0.000027   
4     449  1269.082384     0.000034     0.000018     0.000042     0.000041   
5     478  1662.609925     0.000041     0.000044     0.000040     0.000036   
6     505  1359.525847     0.000025     0.000020     0.000050     0.000051   
7     579  1497.574016     0.000065     0.000034     0.000047     0.000034   
8     607  1281.711753     0.000029     0.000013     0.000036     0.000032   
9     613  1353.701290     0.000048     0.000031     0.000039     0.000047   
10    746  1363.107098     0.000017     0.000010     0.000054     0.000061   
11    792  1488.578922     0.000021     0.000013     0.000044   

In [107]:
#save normalized ones 
adni_gmv_prog.to_csv("/data2/MRI_PET_DATA/graph/csvs/graph_gmv_volumes/mike_gmv_adni_prog_norm.csv", index=False)
adni_gmv_stab.to_csv('/data2/MRI_PET_DATA/graph/csvs/graph_gmv_volumes/mike_gmv_adni_stab_norm.csv', index=False)

In [108]:
adni_gmv_prog = adni_gmv_prog.drop(['RID','TIV','Dataset'], axis=1)
print(adni_gmv_prog.columns)

Index(['vol_l3thVen', 'vol_r3thVen', 'vol_l4thVen', 'vol_r4thVen', 'vol_lAcc',
       'vol_rAcc', 'vol_lAmy', 'vol_rAmy', 'vol_lBst', 'vol_rBst',
       ...
       'vol_lSupParLo', 'vol_rSupParLo', 'vol_lSupTemGy', 'vol_rSupTemGy',
       'vol_lTemPo', 'vol_rTemPo', 'vol_lInfFroAngGy', 'vol_rInfFroAngGy',
       'vol_lTemTraGy', 'vol_rTemTraGy'],
      dtype='object', length=142)


In [109]:
#calculate the means for each row 
# Assuming you have a DataFrame named adni_progs_pruned
# Calculate the mean for each column and create a DataFrame with a single row
mean_values_df = pd.DataFrame(adni_gmv_prog.mean()).T
mean_values_df.index = ['prog_mean']
# Display the new DataFrame
print(mean_values_df)

           vol_l3thVen  vol_r3thVen  vol_l4thVen  vol_r4thVen  vol_lAcc  \
prog_mean     0.000025     0.000021      0.00004     0.000045  0.000224   

           vol_rAcc  vol_lAmy  vol_rAmy  vol_lBst  vol_rBst  ...  \
prog_mean   0.00023  0.000458  0.000452  0.000293  0.000367  ...   

           vol_lSupParLo  vol_rSupParLo  vol_lSupTemGy  vol_rSupTemGy  \
prog_mean       0.005615       0.005477       0.003546       0.003716   

           vol_lTemPo  vol_rTemPo  vol_lInfFroAngGy  vol_rInfFroAngGy  \
prog_mean    0.004861    0.005117          0.001881          0.001901   

           vol_lTemTraGy  vol_rTemTraGy  
prog_mean        0.00072       0.000621  

[1 rows x 142 columns]


In [110]:
adni_gmv_stab_df = adni_gmv_stab.drop(['RID','TIV','Dataset'],axis=1)
stable_mean_values_df = pd.DataFrame(adni_gmv_stab.mean()).T
stable_mean_values_df.index = ['stable_mean']
print(stable_mean_values_df)

                     RID          TIV  vol_l3thVen  vol_r3thVen  vol_l4thVen  \
stable_mean  4011.466667  1465.778199     0.000028     0.000023     0.000039   

             vol_r4thVen  vol_lAcc  vol_rAcc  vol_lAmy  vol_rAmy  ...  \
stable_mean     0.000044  0.000231   0.00024  0.000519  0.000518  ...   

             vol_lSupParLo  vol_rSupParLo  vol_lSupTemGy  vol_rSupTemGy  \
stable_mean       0.005849       0.005726       0.003797       0.003969   

             vol_lTemPo  vol_rTemPo  vol_lInfFroAngGy  vol_rInfFroAngGy  \
stable_mean    0.005238    0.005461           0.00192          0.001999   

             vol_lTemTraGy  vol_rTemTraGy  
stable_mean       0.000754       0.000659  

[1 rows x 144 columns]


In [111]:
#to concat must have list of dataframes 
dfs_to_concat = [mean_values_df,stable_mean_values_df]

means_df = pd.concat(dfs_to_concat)
print(means_df)

             vol_l3thVen  vol_r3thVen  vol_l4thVen  vol_r4thVen  vol_lAcc  \
prog_mean       0.000025     0.000021     0.000040     0.000045  0.000224   
stable_mean     0.000028     0.000023     0.000039     0.000044  0.000231   

             vol_rAcc  vol_lAmy  vol_rAmy  vol_lBst  vol_rBst  ...  \
prog_mean     0.00023  0.000458  0.000452  0.000293  0.000367  ...   
stable_mean   0.00024  0.000519  0.000518  0.000291  0.000357  ...   

             vol_lSupTemGy  vol_rSupTemGy  vol_lTemPo  vol_rTemPo  \
prog_mean         0.003546       0.003716    0.004861    0.005117   
stable_mean       0.003797       0.003969    0.005238    0.005461   

             vol_lInfFroAngGy  vol_rInfFroAngGy  vol_lTemTraGy  vol_rTemTraGy  \
prog_mean            0.001881          0.001901       0.000720       0.000621   
stable_mean          0.001920          0.001999       0.000754       0.000659   

                     RID          TIV  
prog_mean            NaN          NaN  
stable_mean  4011.466667 

In [112]:
#data is nearly identical 
#coul be an issue because I am only passing single values 
t_stat,p_val = stats.ttest_ind(means_df.loc['prog_mean','vol_rSupTemGy'],means_df.loc['stable_mean','vol_rSupTemGy'])
print(t_stat)
print(p_val)
#also not good 

nan
nan


/tmp/ipykernel_2908252/2701945005.py:3: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_stat,p_val = stats.ttest_ind(means_df.loc['prog_mean','vol_rSupTemGy'],means_df.loc['stable_mean','vol_rSupTemGy'])
/data2/MRI_PET_DATA/graph/graph_env/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/data2/MRI_PET_DATA/graph/graph_env/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


In [113]:
#find regions with the largest difference between means 
#iterate through columns to find significant differences 
pd.set_option('display.max_rows', None)
cols = []
p_values = []

for col in means_df.columns:
    prog = means_df.loc['prog_mean',col]
    stab = means_df.loc['stable_mean',col]

    t_stat,p_val = stats.ttest_ind(prog,stab)
    cols.append(col)
    p_values.append(p_val)


# Create a DataFrame to display columns and their p-values
results_df = pd.DataFrame({'Column': cols, 'P-Value': p_values})
print(results_df)
#

                Column  P-Value
0          vol_l3thVen      NaN
1          vol_r3thVen      NaN
2          vol_l4thVen      NaN
3          vol_r4thVen      NaN
4             vol_lAcc      NaN
5             vol_rAcc      NaN
6             vol_lAmy      NaN
7             vol_rAmy      NaN
8             vol_lBst      NaN
9             vol_rBst      NaN
10            vol_lCau      NaN
11            vol_rCau      NaN
12         vol_lExtCbe      NaN
13         vol_rExtCbe      NaN
14          vol_lCbeWM      NaN
15          vol_rCbeWM      NaN
16          vol_lCbrWM      NaN
17          vol_rCbrWM      NaN
18            vol_lCSF      NaN
19            vol_rCSF      NaN
20            vol_lHip      NaN
21            vol_rHip      NaN
22      vol_lInfLatVen      NaN
23      vol_rInfLatVen      NaN
24         vol_lLatVen      NaN
25         vol_rLatVen      NaN
26            vol_lPal      NaN
27            vol_rPal      NaN
28            vol_lPut      NaN
29            vol_rPut      NaN
30      

/tmp/ipykernel_2908252/2220312535.py:11: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_stat,p_val = stats.ttest_ind(prog,stab)


In [114]:
#do t test not on the means but on the original data itself 
# Load your data if not already loaded
# adni_gmv_progs = pd.read_csv('path_to_adni_gmv_progs.csv')
# adni_gmv_stab = pd.read_csv('path_to_adni_gmv_stab.csv')

# Initialize lists or a DataFrame to store your results
cols = []
p_values = []
t_stats = []

# Iterate through columns to perform t-tests
for col in adni_gmv_prog.columns:
    # Get data for this brain region from both datasets
    progs_data = adni_gmv_prog[col]
    stab_data = adni_gmv_stab[col]
    
    # Perform t-test
    t_stat, p_val = stats.ttest_ind(progs_data, stab_data, equal_var=False)  # Assuming variance might not be equal
    
    # Store results
    cols.append(col)
    t_stats.append(t_stat)
    p_values.append(p_val)

# Combine results into a DataFrame for easy viewing and further analysis
results_df = pd.DataFrame({
    'Brain Region': cols,
    'T-Statistic': t_stats,
    'P-Value': p_values
})

# Display or save the results
print(results_df)
# Optionally save to a CSV file
# results_df.to_csv('t_test_results.csv', index=False)


          Brain Region  T-Statistic       P-Value
0          vol_l3thVen    -2.416655  1.622889e-02
1          vol_r3thVen    -2.480124  1.361663e-02
2          vol_l4thVen     0.641240  5.218826e-01
3          vol_r4thVen     0.314144  7.536334e-01
4             vol_lAcc    -2.794471  5.576169e-03
5             vol_rAcc    -3.273217  1.201104e-03
6             vol_lAmy    -7.814149  1.179121e-13
7             vol_rAmy    -8.662730  5.193427e-16
8             vol_lBst     0.202279  8.398325e-01
9             vol_rBst     0.650382  5.159583e-01
10            vol_lCau    -0.963519  3.361898e-01
11            vol_rCau    -1.212236  2.265358e-01
12         vol_lExtCbe    -0.222535  8.240700e-01
13         vol_rExtCbe    -0.259041  7.958095e-01
14          vol_lCbeWM    -1.148506  2.516586e-01
15          vol_rCbeWM    -0.337278  7.361532e-01
16          vol_lCbrWM    -0.822292  4.116700e-01
17          vol_rCbrWM    -0.623718  5.333720e-01
18            vol_lCSF     1.307405  1.921560e-01


In [115]:
#add col where the p val is greater than .05
import pandas as pd
from scipy import stats

# Load your data if not already loaded
# adni_gmv_progs = pd.read_csv('path_to_adni_gmv_progs.csv')
# adni_gmv_stab = pd.read_csv('path_to_adni_gmv_stab.csv')

# Initialize lists or a DataFrame to store your results
cols = []
p_values = []
t_stats = []

# Note: The dataframe should be adni_gmv_progs, not adni_gmv_prog
for col in adni_gmv_prog.columns:  
    # Get data for this brain region from both datasets
    progs_data = adni_gmv_prog[col]
    stab_data = adni_gmv_stab[col]
    
    # Perform t-test
    t_stat, p_val = stats.ttest_ind(progs_data, stab_data, equal_var=False)  # Assuming variance might not be equal
    
    # Store results
    cols.append(col)
    t_stats.append(t_stat)
    p_values.append(p_val)

# Combine results into a DataFrame for easy viewing and further analysis
results_df = pd.DataFrame({
    'Brain Region': cols,
    'T-Statistic': t_stats,
    'P-Value': p_values
})

# Adding the Significant column based on the P-Value
results_df['Significant'] = results_df['P-Value'] < 0.05

# Display or save the results
print(results_df)
# Optionally save to a CSV file
# results_df.to_csv('t_test_results.csv', index=False)

#the resulting negative t stats mean the progs have lower volume than the stab which is good 

          Brain Region  T-Statistic       P-Value  Significant
0          vol_l3thVen    -2.416655  1.622889e-02         True
1          vol_r3thVen    -2.480124  1.361663e-02         True
2          vol_l4thVen     0.641240  5.218826e-01        False
3          vol_r4thVen     0.314144  7.536334e-01        False
4             vol_lAcc    -2.794471  5.576169e-03         True
5             vol_rAcc    -3.273217  1.201104e-03         True
6             vol_lAmy    -7.814149  1.179121e-13         True
7             vol_rAmy    -8.662730  5.193427e-16         True
8             vol_lBst     0.202279  8.398325e-01        False
9             vol_rBst     0.650382  5.159583e-01        False
10            vol_lCau    -0.963519  3.361898e-01        False
11            vol_rCau    -1.212236  2.265358e-01        False
12         vol_lExtCbe    -0.222535  8.240700e-01        False
13         vol_rExtCbe    -0.259041  7.958095e-01        False
14          vol_lCbeWM    -1.148506  2.516586e-01      

In [116]:
#ok so now find 
num_sig = results_df[results_df['Significant']==True].count()
#104 out of 142 are sign diff between groups 
print(num_sig)

Brain Region    104
T-Statistic     104
P-Value         104
Significant     104
dtype: int64


In [117]:
#create list of signifcant regions 
sig_df = results_df[results_df['Significant']==True]
print(sig_df)

          Brain Region  T-Statistic       P-Value  Significant
0          vol_l3thVen    -2.416655  1.622889e-02         True
1          vol_r3thVen    -2.480124  1.361663e-02         True
4             vol_lAcc    -2.794471  5.576169e-03         True
5             vol_rAcc    -3.273217  1.201104e-03         True
6             vol_lAmy    -7.814149  1.179121e-13         True
7             vol_rAmy    -8.662730  5.193427e-16         True
20            vol_lHip    -8.054154  2.567905e-14         True
21            vol_rHip    -8.224218  8.385273e-15         True
23      vol_rInfLatVen    -2.190368  2.932874e-02         True
24         vol_lLatVen    -3.434060  6.744006e-04         True
25         vol_rLatVen    -3.501431  5.337273e-04         True
28            vol_lPut    -2.906063  3.929864e-03         True
29            vol_rPut    -3.379108  8.179681e-04         True
30         vol_lThaPro    -2.029698  4.327452e-02         True
31         vol_rThaPro    -2.712850  7.066902e-03      

In [118]:
#read paper on strucutral covaruance
#could always make the case that for some reason this is happening because of the MCI swelling that happens early in mci 

#I need to normalize these values by how big the region is supposed to be because larger regions will have larger differences 

#this assumes that the stable is bigger than the prog (so positive values mean the stable patients have larger brain volumes)

# Calculate the difference between 'stable_mean' and 'prog_mean' for each column
difference_df = means_df.loc['stable_mean'] - means_df.loc['prog_mean']

# Create a new DataFrame with two columns: column name and difference
result_df = pd.DataFrame({'Column Name': difference_df.index, 'Difference': difference_df.values})

# Sort the DataFrame by the absolute difference in descending order
result_df['Absolute Difference'] = abs(result_df['Difference'])
result_df = result_df.sort_values(by='Absolute Difference', ascending=False)

# Reset the index for a cleaner display
result_df.reset_index(drop=True, inplace=True)

# Display the result DataFrame
print(result_df)


           Column Name    Difference  Absolute Difference
0        vol_rMidTemGy  6.253435e-04         6.253435e-04
1        vol_lMidTemGy  6.154075e-04         6.154075e-04
2        vol_rInfTemGy  5.502337e-04         5.502337e-04
3        vol_lInfTemGy  4.820451e-04         4.820451e-04
4           vol_rAngGy  4.211321e-04         4.211321e-04
5        vol_rMidFroGy  4.079100e-04         4.079100e-04
6           vol_lAngGy  3.822960e-04         3.822960e-04
7           vol_lFusGy  3.782708e-04         3.782708e-04
8           vol_lTemPo  3.773545e-04         3.773545e-04
9           vol_rFusGy  3.741495e-04         3.741495e-04
10            vol_rPCu  3.662879e-04         3.662879e-04
11       vol_lMidFroGy  3.553572e-04         3.553572e-04
12          vol_rTemPo  3.440670e-04         3.440670e-04
13       vol_lSupFroGy  3.285975e-04         3.285975e-04
14            vol_lPCu  3.225442e-04         3.225442e-04
15       vol_rSupFroGy  3.079183e-04         3.079183e-04
16          vo

In [1]:
#now do the same for nacc 
#combo nacc with both prog and stab
print(comb_gmv_nacc)

NameError: name 'comb_gmv_nacc' is not defined